In [1]:
import EmbeddedCV
import cv2
import numpy as np
import os
from scipy.ndimage import convolve
from matplotlib import pyplot as plt
import QOI

# Test verifiers

In [2]:
import numpy as np

def check_correctness(y_pred, y):
    abs_diff = np.abs(y_pred.astype(np.int32) - y.astype(np.int32))
    max_diff = np.max(abs_diff)
    avg_diff = np.mean(abs_diff)

    # Print the indices where the absolute difference is larger than 1
    diff_unacceptable = np.where(abs_diff > 5)
    # print(f"Shape of diff_unacceptable {diff_unacceptable.shape}")
    for idx in zip(*diff_unacceptable):
        print(idx)

    if y_pred.shape[2] == 1:
        gray = True
    else: 
        gray = False

    cv2.imwrite("pred.png", y_pred)
    cv2.imwrite("true.png", y)
    cv2.imwrite("diff.png", abs_diff)

    print("Max diff:", max_diff)
    print("Avg diff:", avg_diff)

    return True

# Naive implementations

Gaussian blur

In [3]:
from functions import gaussian_blur, canny_edge, sobel_min_max, median_blur

# Tests

Setup

In [4]:
channels = 3
rows = 576
cols = 768

cam = EmbeddedCV.CamSimulator(channels, rows, cols)
host = EmbeddedCV.Host(cam, channels, rows, cols)

Test cases

In [5]:
image_path = "Background_Subtraction_Tutorial_frame_1.png"
image = cv2.imread(image_path)

flat_image = image.flatten()

cam.StoreData(flat_image)

Sanity Check

In [6]:
# host_output = np.zeros(shape=(channels*rows*cols), dtype=np.uint8)

# host.Sanity(host_output)
# host_output = host_output.reshape((rows, cols, channels))

# naive_output = image

# check_correctness(y_pred=host_output, y=naive_output)

MedianBlur

In [7]:
# kernel_height, kernel_width = 3, 3

# host_output = np.zeros(shape=(channels*rows*cols), dtype=np.uint8)

# host.MedianBlur(host_output, kernel_height, kernel_width)
# host_output = host_output.reshape((rows, cols, channels))

# naive_output = median_blur(image, kernel_height, kernel_width)

# check_correctness(y_pred=host_output, y=naive_output)

GaussianBlur

In [8]:
# kernel_height, kernel_width = 3, 3
# sigma_x, sigma_y = 0, 0

# host_output = np.zeros(shape=(channels*rows*cols), dtype=np.uint8)

# host.GaussianBlur(host_output, kernel_height, kernel_width, sigma_x, sigma_y)
# host_output = host_output.reshape((rows, cols, channels))

# naive_output = gaussian_blur(image, kernel_height, kernel_width, sigma_x, sigma_y)

# check_correctness(y_pred=host_output, y=naive_output)

Sobel

In [9]:
host_output = np.zeros(shape=(channels*rows*cols), dtype=np.uint8)

host.Sobel(host_output)
host_output = host_output.reshape((rows, cols, channels))

naive_output = sobel_min_max(image)

check_correctness(y_pred=host_output, y=naive_output)

(0, 92, 0)
(0, 92, 1)
(0, 92, 2)
(0, 93, 0)
(0, 93, 1)
(0, 93, 2)
(0, 108, 0)
(0, 108, 1)
(0, 108, 2)
(0, 109, 0)
(0, 109, 1)
(0, 109, 2)
(0, 110, 0)
(0, 110, 1)
(0, 110, 2)
(0, 112, 2)
(0, 146, 2)
(0, 159, 1)
(0, 159, 2)
(0, 160, 1)
(0, 160, 2)
(0, 161, 0)
(0, 161, 1)
(0, 162, 1)
(0, 164, 0)
(0, 164, 1)
(0, 164, 2)
(0, 166, 0)
(0, 166, 1)
(0, 166, 2)
(0, 168, 0)
(0, 168, 1)
(0, 168, 2)
(0, 169, 1)
(0, 180, 1)
(0, 180, 2)
(0, 186, 0)
(0, 186, 1)
(0, 186, 2)
(0, 187, 0)
(0, 187, 1)
(0, 187, 2)
(0, 188, 0)
(0, 188, 1)
(0, 188, 2)
(0, 189, 0)
(0, 189, 1)
(0, 189, 2)
(0, 190, 0)
(0, 190, 1)
(0, 190, 2)
(0, 191, 0)
(0, 191, 1)
(0, 191, 2)
(0, 192, 0)
(0, 192, 1)
(0, 192, 2)
(0, 193, 0)
(0, 193, 1)
(0, 193, 2)
(0, 194, 0)
(0, 194, 1)
(0, 194, 2)
(0, 195, 0)
(0, 195, 1)
(0, 195, 2)
(0, 196, 0)
(0, 196, 1)
(0, 196, 2)
(0, 197, 0)
(0, 197, 1)
(0, 197, 2)
(0, 198, 0)
(0, 198, 1)
(0, 199, 0)
(0, 199, 1)
(0, 199, 2)
(0, 202, 0)
(0, 202, 1)
(0, 202, 2)
(0, 203, 0)
(0, 203, 1)
(0, 203, 2)
(0, 204, 0

True

CannyEdge

In [10]:
# lowThreshold = 100
# highThreshold = 200

# host_output = np.zeros(shape=(1*rows*cols), dtype=np.uint8)
 
# host.CannyEdge(host_output, lowThreshold, highThreshold)
# host_output = host_output.reshape((rows, cols, 1))

# naive_output = canny_edge(image, lowThreshold, highThreshold)

# check_correctness(y_pred=host_output, y=naive_output)

QOI

In [11]:
# q = QOI(flat_image, channels, rows, cols)
# q()